In [1]:
########################################################################
# File       : Read_and_Display_Images_in_Napari.ipynb
# Version    : 0.3
# Author     : czsrh
# Date       : 26.11.2020
# Institution : Carl Zeiss Microscopy GmbH
#
# Disclaimer: Just for testing - Use at your own risk.
# Feedback or Improvements are welcome.
########################################################################

***Import Libaries to run this notebook***

* [PyPi - czifile](https://pypi.org/project/czifile/)

Thanks to Christoph Gohlke. For details and the source code please see [czifile.py](https://www.lfd.uci.edu/~gohlke/code/czifile.py.html)

* [PyPi - AICSImageIO](https://pypi.org/project/aicsimageio/)

Library to read OME-TIFF and CZI images. It uses the powerful [libCZI](https://github.com/zeiss-microscopy/libCZI) C++ library to read CZI images.

* [PyPi - napari](https://pypi.org/project/napari/)

Very new, fast and cool multi-dimensional array viewer. For more details see the prject page [GitHub - Napari](https://github.com/napari/napari)

* imagefileutils

This is collection of of usefule function to read CZI and OME-TFF image files. This tool uses many of the modules mentioned above

***

Another important modules

* [PyPi - xmltodict](https://pypi.org/project/xmltodict/)

This is turns an XML, e.g. the OME-XML or CZI metadata into an "easy-to-use" python dictionary.

In [2]:
# this can be used to switch on/off warnings
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

import sys
sys.path.append(r'modules')
import imgfileutils as imf

# import the libraries mentioned above
import czifile as zis
from aicsimageio import AICSImage
import xmltodict
import os
import numpy as np
import napari
import xml.etree.ElementTree as ET
from IPython.display import display, HTML

In [3]:
# define your testfiles here

#filename = r'testdata\Nucleus_Mito.czi'
filename = r'testdata\CellDivision_T=10_Z=15_CH=2_DCV_small.czi'
#filename = r'testdata\3CH_Z-Stack_RN-DAPI.ome.tiff'

In [4]:
# !!! Only use for CZI files !!! 

# To read the CZI metadata one can use the czifile functionality

# get czi object using the filename
czi = zis.CziFile(filename)

# get the metadata and close the czi file object
mdczi = czi.metadata()
czi.close()

# parse the XML into a dictionary
metadatadict_czi = xmltodict.parse(mdczi)

In [5]:
# !!! Only use for CZI files !!! 

# Sometimes it can be very usefule to save the CZI metadate as an actual XML file.
# This can be done quite easily by converting the metadata string into an XML tree.
# This tree object can be directly as an XML file

# define the new filename for the XML to be created later
xmlfile = filename.replace('.czi', '_CZI_MetaData.xml')

# get the element tree
tree = ET.ElementTree(ET.fromstring(mdczi))

# write xml to disk
tree.write(xmlfile, encoding='utf-8', method='xml')

print('Write special CZI XML metainformation for: ', xmlfile)

Write special CZI XML metainformation for:  testdata\CellDivision_T=10_Z=15_CH=2_DCV_small_CZI_MetaData.xml


In [6]:
# !!! Only use for CZI files !!! 

# or much shorter
xmlczi = imf.writexml_czi(filename)
print(xmlczi)

testdata\CellDivision_T=10_Z=15_CH=2_DCV_small_CZI_MetaData.xml


### Remark

As one can clearly see there are a lot of metadata not all of them are needed for every workflow.

In [7]:
# get the array and the metadata
metadata, add_metadata = imf.get_metadata(filename)
img = AICSImage(filename)
array = img.get_image_data()

Detected Image Type (based on extension):  czi
Trying to extract Scene and Well information if existing ...
No valid Scene or Well information found: 'S'
Scales will not be rounded.


In [8]:
# shape of numpy array
print('Array Shape                       : ', array.shape)

# dimension order from metadata
print('Dimension Order (BioFormats)      : ', metadata['DimOrder BF Array'])

# shape and dimension entry from CZI file as returned by czifile.py
print('CZI Array Shape (czifile)         : ', metadata['Shape_czifile'])
print('CZI Dimension Entry (czifile)     : ', metadata['Axes_czifile'])
print('CZI Array Shape (AICSImageIO)     : ', metadata['Shape_aics'])
print('CZI Dimension Entry (AICSImageIO) : ', metadata['Axes_aics'])

Array Shape                       :  (1, 10, 2, 15, 256, 256)
Dimension Order (BioFormats)      :  None
CZI Array Shape (czifile)         :  (1, 10, 2, 15, 256, 256, 1)
CZI Dimension Entry (czifile)     :  BTCZYX0
CZI Array Shape (AICSImageIO)     :  (1, 10, 2, 15, 256, 256)
CZI Dimension Entry (AICSImageIO) :  STCZYX


In [9]:
# show dimensions and scaling
print('SizeT : ', metadata['SizeT'])
print('SizeZ : ', metadata['SizeZ'])
print('SizeC : ', metadata['SizeC'])
print('SizeX : ', metadata['SizeX'])
print('SizeY : ', metadata['SizeY'])
print('XScale: ', metadata['XScale'])
print('YScale: ', metadata['YScale'])
print('ZScale: ', metadata['ZScale'])

SizeT :  10
SizeZ :  15
SizeC :  2
SizeX :  256
SizeY :  256
XScale:  0.091
YScale:  0.091
ZScale:  0.32


In [10]:
### show complete metadata

# convert metadata dictionary to a pandas dataframe
mdframe = imf.md2dataframe(metadata)

# and display it nicely as a HTML inside the jupyter notebook
display(HTML(mdframe.to_html()))

,Parameter,Value
0,Directory,testdata
1,Filename,CellDivision_T=10_Z=15_CH=2_DCV_small.czi
2,Extension,czi
3,ImageType,czi
4,AcqDate,2016-02-12T09:41:02.4915604Z
5,TotalSeries,None
6,SizeX,256
7,SizeY,256
8,SizeZ,15
9,SizeC,2


In [ ]:
# configure napari automatically based on metadata and show the OME-TIFF stack
layers = imf.show_napari(array, metadata,
                         blending='additive',
                         gamma=0.85,
                         add_mdtable=True,
                         rename_sliders=True)

Initializing Napari Viewer ...


Extract Channel as NumPy.Array
Adding Channel  :  AF555
Shape Channel   :  0 (1, 10, 15, 256, 256)
Scaling Factors :  [1.0, 1.0, 3.516, 1.0, 1.0]
Scaling:  0.0 4927.0
Extract Channel as NumPy.Array
Adding Channel  :  AF488
Shape Channel   :  1 (1, 10, 15, 256, 256)
Scaling Factors :  [1.0, 1.0, 3.516, 1.0, 1.0]
Scaling:  0.0 25331.0
Renaming the Sliders based on the Dimension String ....


jupyter nbconvert Read_and_Display_Images_in_Napari.ipynb --to slides --post serve